# PYTHON PROGRAMMING - FUNDAMENTAL


In [1]:
###############################################################
# Customer Segmentation with RFM
###############################################################

# Customer Segmentation with RFM in 6 Steps

# 1. Business Problem
# 2. Data Understanding
# 3. Data Preparation
# 4. Calculating RFM Metrics
# 5. Calculating RFM Scores
# 6. Naming & Analysing RFM Segments




In [2]:
###############################################################
# Data Understanding
###############################################################

import datetime as dt
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
%%time
df_ = pd.read_excel("online_retail_II.xlsx",sheet_name="Year 2009-2010")

CPU times: user 3min 13s, sys: 829 ms, total: 3min 14s
Wall time: 3min 14s


In [4]:
df_.shape

(525461, 8)

In [5]:
#df_test = df_[df_["Country"] == 'United Kingdom']
#df_test.shape

In [6]:
df = df_.copy()
df = df[df["Country"] == 'United Kingdom']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [7]:
df.shape

(485852, 8)

In [8]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    106429
Country             0
dtype: int64

In [9]:
# What is the number of unique products?
df["Description"].nunique()

4657

In [10]:
# how many of which products do you have?
df["Description"].value_counts().head()

WHITE HANGING HEART T-LIGHT HOLDER    3433
REGENCY CAKESTAND 3 TIER              2005
STRAWBERRY CERAMIC TRINKET BOX        1730
ASSORTED COLOUR BIRD ORNAMENT         1381
HOME BUILDING BLOCK WORD              1347
Name: Description, dtype: int64

In [11]:
# Which is the most ordered product?
df.groupby("Description").agg({"Quantity": "sum"}).head()

,Quantity
Description,
21494,-720
22467,-2
22719,2
DOORMAT UNION JACK GUNS AND ROSES,177
3 STRIPEY MICE FELTCRAFT,629


In [12]:
# How do we sort the above output?
df.groupby("Description").agg({"Quantity": "sum"}).sort_values("Quantity", ascending=False).head()

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,52252
WORLD WAR 2 GLIDERS ASSTD DESIGNS,51434
BROCADE RING PURSE,47495
ASSORTED COLOUR BIRD ORNAMENT,41855
PACK OF 72 RETRO SPOT CAKE CASES,36393


In [13]:
# how many invoices have been issued?
df["Invoice"].nunique()

26633

In [14]:
#How many Row per invoices?
df.shape[0]/df["Invoice"].nunique()

18.242481132429692

In [15]:
df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
196,C489459,90200A,PURPLE SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
197,C489459,90200D,PINK SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
198,C489459,90200B,BLACK SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
199,C489459,90200E,GREEN SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom
200,C489459,90200C,BLUE SWEETHEART BRACELET,-3,2009-12-01 10:44:00,4.25,17592.0,United Kingdom


In [16]:
# Average revenue per invoice?
# (it is necessary to create a new variable by multiplying two variables)
# Let's create the df again by subtracting the returns
df = df[~df["Invoice"].str.contains("C", na=False)]

In [17]:
df[df["Invoice"].str.contains("C", na=False)].head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country


In [18]:
# Which is the most ordered product?
#df.groupby("Description").agg({"Quantity": "sum"}).head()

In [19]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

In [20]:
# what are the most expensive products?
df.sort_values("Price", ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
241827,512771,M,Manual,1,2010-06-17 16:53:00,25111.09,NaN,United Kingdom,25111.09
517955,537632,AMAZONFEE,AMAZON FEE,1,2010-12-07 15:08:00,13541.33,NaN,United Kingdom,13541.33
135013,502263,M,Manual,1,2010-03-23 15:22:00,10953.50,12918.0,United Kingdom,10953.50
135015,502265,M,Manual,1,2010-03-23 15:28:00,10953.50,NaN,United Kingdom,10953.50
358639,524159,M,Manual,1,2010-09-27 16:12:00,10468.80,14063.0,United Kingdom,10468.80


In [21]:
# How many orders came from which country?
df["Country"].value_counts()

United Kingdom    477058
Name: Country, dtype: int64

In [22]:
# how much income did UK generate?
df.groupby("Country").agg({"TotalPrice": "sum"}).sort_values("TotalPrice", ascending=False).head()

,TotalPrice
Country,
United Kingdom,8.709204e+06


In [23]:
###############################################################
# Data Preparation
###############################################################
df.dropna(inplace=True)

In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,370951.0,11.994444,67.394793,1.0,2.00,4.00,12.00,10000.0
Price,370951.0,3.145220,30.551482,0.0,1.25,1.95,3.75,10953.5
Customer ID,370951.0,15565.407693,1593.187209,12346.0,14215.00,15601.00,16940.00,18287.0
TotalPrice,370951.0,19.988505,73.930472,0.0,4.25,10.50,17.85,15818.4


In [25]:
df.describe([0.01, 0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95, 0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,95%,99%,max
Quantity,370951.0,11.994444,67.394793,1.0,1.00,1.00,1.00,2.00,4.00,12.00,24.00,36.0,120.00,10000.0
Price,370951.0,3.145220,30.551482,0.0,0.29,0.42,0.65,1.25,1.95,3.75,6.75,8.5,12.75,10953.5
Customer ID,370951.0,15565.407693,1593.187209,12346.0,12835.00,13052.00,13283.00,14215.00,15601.00,16940.00,17769.00,17930.0,18215.00,18287.0
TotalPrice,370951.0,19.988505,73.930472,0.0,0.65,1.25,1.95,4.25,10.50,17.85,34.00,61.2,175.20,15818.4


In [26]:
###############################################################
# Calculating RFM Metrics
###############################################################

# Recency, Frequency, Monetary


# Today's date - Last purchase

print(df["InvoiceDate"].max())

today_date = dt.datetime(2010, 12, 11)
print(today_date)


#                                'Invoice': [lambda num: num.nunique(),lambda num: len(num)],
        
        
rfm = df.groupby('Customer ID').agg({'InvoiceDate': lambda date: (today_date - date.max()).days,
                                     'Invoice': lambda num: len(num),
                                     'TotalPrice': lambda TotalPrice: TotalPrice.sum()})


rfm.columns = ['Recency', 'Frequency', 'Monetary']

rfm = rfm[(rfm["Monetary"]) > 0 & (rfm["Frequency"] > 0)]

rfm.head()

2010-12-09 20:01:00
2010-12-11 00:00:00


,Recency,Frequency,Monetary
Customer ID,,,
12346.0,165,33,372.86
12608.0,40,16,415.79
12745.0,122,22,723.85
12746.0,176,17,254.55
12747.0,5,154,5080.53


In [27]:
###############################################################
# Calculating RFM Scores
###############################################################

# Recency
rfm["RecencyScore"] = pd.qcut(rfm['Recency'], 5, labels=[5, 4, 3, 2, 1])

rfm["FrequencyScore"] = pd.qcut(rfm['Frequency'], 5, labels=[1, 2, 3, 4, 5])
#rfm["FrequencyScore"]= pd.qcut(rfm["Frequency"].rank(method="first"),5, labels=[1,2,3,4,5])

rfm["MonetaryScore"] = pd.qcut(rfm['Monetary'], 5, labels=[1, 2, 3, 4, 5])


rfm["RFM_SCORE"] = (rfm['RecencyScore'].astype(str) +
                    rfm['FrequencyScore'].astype(str) +
                    rfm['MonetaryScore'].astype(str))


rfm[rfm["RFM_SCORE"] == "555"].head()


,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12747.0,5,154,5080.53,5,5,5,555
12748.0,1,2634,22879.66,5,5,5,555
12838.0,9,300,2715.35,5,5,5,555
12839.0,3,262,4480.31,5,5,5,555
12841.0,5,282,3518.33,5,5,5,555


In [28]:
rfm['RecencyScore'].value_counts()

5    836
1    794
4    790
2    788
3    761
Name: RecencyScore, dtype: int64

In [29]:
rfm['RFM_SCORE'].value_counts()

555    275
111    230
455    148
122    129
211    119
544    107
233    100
344     94
322     91
444     91
222     80
311     77
333     71
121     71
545     70
355     69
433     68
244     64
422     62
411     62
112     61
554     60
454     56
133     54
223     53
212     48
131     45
533     45
132     41
323     41
      ... 
235      8
324      8
214      8
213      8
514      7
155      7
114      7
145      6
314      5
115      5
415      5
414      5
124      5
215      4
153      4
135      3
125      3
315      3
441      3
341      3
525      2
452      2
352      2
425      2
325      2
515      2
241      2
141      1
541      1
152      1
Name: RFM_SCORE, Length: 118, dtype: int64

In [30]:
rfm[rfm["RFM_SCORE"] == "111"]

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE
Customer ID,,,,,,,
12821.0,247,7,128.08,1,1,1,111
12829.0,186,8,92.30,1,1,1,111
12846.0,318,1,15.58,1,1,1,111
12861.0,255,4,68.25,1,1,1,111
12925.0,228,5,35.28,1,1,1,111
12934.0,372,2,181.65,1,1,1,111
12954.0,274,10,162.45,1,1,1,111
12969.0,238,5,122.45,1,1,1,111
12992.0,361,6,93.35,1,1,1,111


In [31]:
###############################################################
# Naming & Analysing RFM Segments
###############################################################

# RFM isimlendirmesi
seg_map = {
    r'[1-2][1-2]': 'Hibernating',
    r'[1-2][3-4]': 'At_Risk',
    r'[1-2]5': 'Cant_Loose',
    r'3[1-2]': 'About_to_Sleep',
    r'33': 'Need_Attention',
    r'[3-4][4-5]': 'Loyal_Customers',
    r'41': 'Promising',
    r'51': 'New_Customers',
    r'[4-5][2-3]': 'Potential_Loyalists',
    r'5[4-5]': 'Champions'
}



In [32]:
rfm['Segment'] = rfm['RecencyScore'].astype(str) + rfm['FrequencyScore'].astype(str)
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,165,33,372.86,2,3,2,232,23
12608.0,40,16,415.79,3,2,2,322,32
12745.0,122,22,723.85,2,2,3,223,22
12746.0,176,17,254.55,2,2,2,222,22
12747.0,5,154,5080.53,5,5,5,555,55


In [33]:
rfm['Segment'].value_counts()

55    362
11    315
12    239
54    229
45    224
23    205
34    193
22    191
21    187
44    182
32    169
13    165
43    152
33    147
24    137
31    136
42    130
53    117
35    116
41    102
52     72
25     68
51     56
14     54
15     21
Name: Segment, dtype: int64

In [34]:
rfm['Segment'] = rfm['Segment'].replace(seg_map, regex=True)
df[["Customer ID"]].nunique()

Customer ID    3971
dtype: int64

In [35]:
rfm.head()

,Recency,Frequency,Monetary,RecencyScore,FrequencyScore,MonetaryScore,RFM_SCORE,Segment
Customer ID,,,,,,,,
12346.0,165,33,372.86,2,3,2,232,At_Risk
12608.0,40,16,415.79,3,2,2,322,About_to_Sleep
12745.0,122,22,723.85,2,2,3,223,Hibernating
12746.0,176,17,254.55,2,2,2,222,Hibernating
12747.0,5,154,5080.53,5,5,5,555,Champions


In [36]:
def get_std(x): 
    return np.std(x)

In [37]:
import numpy as np
rfm[["Segment", "Recency", "Frequency", "Monetary"]].groupby("Segment").agg(["mean", "count", np.std ])
#rfm.columns = ['1', '2', '3','4','5','6','7','8']
#rfm.columns = df.index.set_levels(['b1','c1','f1'],level=1)
#rfm

Recency                    Frequency        \
                           mean count        std        mean count   
Segment                                                              
About_to_Sleep        53.308197   305   9.895345   15.800000   305   
At_Risk              163.891266   561  81.077524   57.586453   561   
Cant_Loose           130.550562    89  53.161634  227.752809    89   
Champions              7.006768   591   4.507616  260.189509   591   
Hibernating          205.640558   932  90.349129   14.052575   932   
Loyal_Customers       37.051748   715  16.060420  170.330070   715   
Need_Attention        52.510204   147   9.528161   44.795918   147   
New_Customers          7.910714    56   4.387075    7.071429    56   
Potential_Loyalists   18.501062   471   9.652444   36.303609   471   
Promising             25.215686   102   5.886093    8.029412   102   

                                    Monetary                      
                            std         mean count           std  
Segment                                                           
About_to_Sleep         8.364792   449.083607   305    575.273243  
At_Risk               23.207760  1053.429611   561   1632.253291  
Cant_Loose           159.406733  2689.745067    89   2387.632522  
Champions            368.914171  6018.414743   591  17884.635641  
Hibernating            8.557785   443.556804   932   1224.452168  
Loyal_Customers      152.359979  2553.726708   715   3183.239565  
Need_Attention         8.154282   910.731565   147    788.359696  
New_Customers          3.962814   490.808750    56    629.338343  
Potential_Loyalists   13.182321   939.676157   471   2185.245001  
Promising              3.865193   463.541078   102    781.279937

In [38]:
rfm[rfm["Segment"] == "Need_Attention"].head()
rfm[rfm["Segment"] == "Need_Attention"].index

Float64Index([12903.0, 12987.0, 12989.0, 13082.0, 13195.0, 13233.0, 13241.0,
              13253.0, 13296.0, 13299.0,
              ...
              17834.0, 18016.0, 18078.0, 18090.0, 18110.0, 18112.0, 18136.0,
              18141.0, 18241.0, 18247.0],
             dtype='float64', name='Customer ID', length=147)

In [39]:
new_df = pd.DataFrame()

In [40]:
new_df["Need_Attention"] = rfm[rfm["Segment"] == "Need_Attention"].index

In [41]:
new_df.to_csv("Need_Attention.csv")